<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/happywhale_ensemble_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/mydrive', force_remount=True)

Mounted at /content/mydrive


In [14]:
!cp /content/mydrive/MyDrive/kaggle/sub*.csv /content/

In [15]:
import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
 
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras import backend as K
 
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle
import json
import tensorflow_hub as tfhub
from datetime import datetime

In [16]:
b6_sub=  pd.read_csv('/content/submission729.csv',index_col='image') 

In [17]:
b6_sub.head()

,predictions
image,
d846a86edded63.jpg,03a3bbaeed84 new_individual a977dca03f23 4b6a2...
7a785b700b0339.jpg,c93996835aa8 new_individual d36d5a07500f 76afa...
50df0a954eb94c.jpg,713eb1a00c3d new_individual 8c78e0a8abee 9a8be...
7d0f46dd5f4108.jpg,982224d06f8c new_individual 3d5a4dd24d21 f63a9...
0f167a46fa2688.jpg,6989dcf1533f new_individual bb9890db1425 a5162...


In [18]:
b7_sub=  pd.read_csv('/content/submission754.csv',index_col='image') 

In [19]:
b7_sub.head()

,predictions
image,
a3f60cad2aae2d.jpg,e8ac23241df5 new_individual 2c6dd2efff3b b602f...
dd806b5d0f42e1.jpg,13e453fd9598 new_individual 91dc9523c890 bc1eb...
80d34a911209bc.jpg,47a2f9918aa2 new_individual 4c4f863197af 5a3d2...
7a785b700b0339.jpg,c93996835aa8 new_individual bef01de5aacc d4917...
0f167a46fa2688.jpg,6989dcf1533f new_individual 942255cacac0 6c019...


In [20]:
df6=b6_sub.sort_values(['image']).reset_index()

In [21]:
df7=b7_sub.sort_values(['image']).reset_index()

In [22]:
df6.head()

,image,predictions
0,000110707af0ba.jpg,fbe2b15b5481 new_individual 574bd81ec992 e4fbd...
1,0006287ec424cb.jpg,1424c7fec826 new_individual e72a5024c4ad f0efb...
2,000809ecb2ccad.jpg,1ce3ba6a3c29 new_individual d3cedf439859 b8125...
3,00098d1376dab2.jpg,c4274d90be60 new_individual 164b7f79dcb7 255ae...
4,000b8d89c738bd.jpg,cf500d3874bc new_individual ef2bf7f7553d 59ce2...


In [23]:
df7.head()

,image,predictions
0,000110707af0ba.jpg,fbe2b15b5481 new_individual 574bd81ec992 e4fbd...
1,0006287ec424cb.jpg,1424c7fec826 new_individual 5f0ae1184916 c3b7d...
2,000809ecb2ccad.jpg,1ce3ba6a3c29 new_individual d3cedf439859 b8125...
3,00098d1376dab2.jpg,c4274d90be60 new_individual 164b7f79dcb7 255ae...
4,000b8d89c738bd.jpg,cf500d3874bc new_individual 4d18b20adff3 53741...


In [24]:
df7.columns

Index(['image', 'predictions'], dtype='object')

In [25]:
sub=pd.DataFrame(columns=['image', 'predictions'],index=None)
sub

,image,predictions


In [26]:
sub=df6.copy(deep=True)

In [27]:
len(df6)

27956

In [28]:
'''
>>> results_list = [[1,2,3], [1,2,4]]
>>> results_union = set().union(*results_list)
>>> print(results_union)
set([1, 2, 3, 4])
'''

'\nresults_list = [[1,2,3], [1,2,4]]\nresults_union = set().union(*results_list)\nprint(results_union)\nset([1, 2, 3, 4])\n'

In [29]:
'''
dict1 = {1: 1, 2: 9, 3: 4}
sorted_tuples = sorted(dict1.items(), key=lambda item: item[1])
print(sorted_tuples)  # [(1, 1), (3, 4), (2, 9)]
sorted_dict = {k: v for k, v in sorted_tuples}

print(sorted_dict)  # {1: 1, 3: 4, 2: 9}


>>> s = [2, 3, 1, 4, 5, 3]
>>> sorted(range(len(s)), key=lambda k: s[k])
[2, 0, 1, 5, 3, 4]

'''

'\ndict1 = {1: 1, 2: 9, 3: 4}\nsorted_tuples = sorted(dict1.items(), key=lambda item: item[1])\nprint(sorted_tuples)  # [(1, 1), (3, 4), (2, 9)]\nsorted_dict = {k: v for k, v in sorted_tuples}\n\nprint(sorted_dict)  # {1: 1, 3: 4, 2: 9}\n\n\n>>> s = [2, 3, 1, 4, 5, 3]\n>>> sorted(range(len(s)), key=lambda k: s[k])\n[2, 0, 1, 5, 3, 4]\n\n'

In [30]:
for i in df6.index:
  p6=[x for x in df6['predictions'][i].split()] 
      
   
  p7=[x for x in df7['predictions'][i].split()]

  p=set().union(*[p6,p7])
  p=list(p)
  wt=[1,1/2,1/3,1/4,1/5]
  pv=[0 for x in range(len(p))]
  for j in range(len(p)):
      v=p[j]
      v6,v7=0,0 
      for k in range(5):
        if p6[k]==v:
          v6=wt[k]
        if p7[k]==v:
          v7=wt[k]
      pv[j]=v6+v7

  pvs=sorted(range(len(pv)), key=lambda k: pv[k],reverse=True)
  #pvs=np.argsort(pv)

  '''
  print(p6)
  #print('\n')
  print(p7)
  print(p)
  print(pv)
  print(pvs)
  '''
  
  
  wtp=[p[idx] for idx in pvs]
  subp=wtp[:5]
  
  print('\n')
  print(i)
  print(wtp)
  print(subp)
  print('\n\n')

  sub['image'][i]=df6['image'][i]
  sub['predictions'][i]=' '.join(x for x in subp)

  #if i==3:
  #  break

sub.to_csv('submission.csv',index=False)
sub.head()




Streaming output truncated to the last 5000 lines.


27331
['bbeac4b2964e', 'new_individual', '768194a67cb8', '88337c2f4b44', '6949b3b9b3eb', '3e1546ac6905', '938b7e931166']
['bbeac4b2964e', 'new_individual', '768194a67cb8', '88337c2f4b44', '6949b3b9b3eb']





27332
['2ae339a65376', 'new_individual', '6a6fa3ec3810', '1a4943f84c5e', 'ef2002ea8327']
['2ae339a65376', 'new_individual', '6a6fa3ec3810', '1a4943f84c5e', 'ef2002ea8327']





27333
['55feac3c8d61', 'new_individual', '075a008b8a47', '6aefbe581e8c', '17ae98f695f2']
['55feac3c8d61', 'new_individual', '075a008b8a47', '6aefbe581e8c', '17ae98f695f2']





27334
['6c72e287a822', 'new_individual', '78e927d4da1c', '8ba67ef3c591', '40a43a04b891', '938b7e931166', '48936da899c3', '0c59b7b50fa6']
['6c72e287a822', 'new_individual', '78e927d4da1c', '8ba67ef3c591', '40a43a04b891']





27335
['new_individual', 'b3fd1f9a25a7', 'bbd8812fb038', '3eaac33d5a64', '851a09c20be2', '93c725b020eb', '6622bb960c8e', '171aa23102f3', '5b22246a675f']
['new_

,image,predictions
0,000110707af0ba.jpg,fbe2b15b5481 new_individual 574bd81ec992 e4fbd...
1,0006287ec424cb.jpg,1424c7fec826 new_individual 5f0ae1184916 e72a5...
2,000809ecb2ccad.jpg,1ce3ba6a3c29 new_individual d3cedf439859 b8125...
3,00098d1376dab2.jpg,c4274d90be60 new_individual 164b7f79dcb7 255ae...
4,000b8d89c738bd.jpg,cf500d3874bc new_individual ef2bf7f7553d 4d18b...
